# Préparation et importation des bibliothèques

In [8]:
import numpy as np
import pandas as pd
from numpy.random import choice, normal, uniform, binomial
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import random

# Définition des constantes statistiques

In [9]:
# Nombre d'enregistrements
NB_ENREGISTREMENTS = 40000

# Répartition urbain/rural
POURCENTAGE_URBAIN = 0.65  # Approximativement 65% urbain, 35% rural

# Revenus moyens par milieu
REVENU_MOYEN = 21949
REVENU_MOYEN_URBAIN = 26988
REVENU_MOYEN_RURAL = 12862

# Écart-type des revenus (à ajuster pour une distribution réaliste)
ECART_TYPE_URBAIN = 15000
ECART_TYPE_RURAL = 7000

# Pourcentage de personnes gagnant moins que la moyenne
POURCENTAGE_MOINS_MOYENNE = 0.718
POURCENTAGE_MOINS_MOYENNE_URBAIN = 0.659
POURCENTAGE_MOINS_MOYENNE_RURAL = 0.854

# Définition des fonctions génératrices

In [10]:
def generer_age():
    """Génère un âge entre 18 et 80 ans avec une distribution réaliste."""
    # Distribution plus dense entre 25 et 55 ans
    return int(np.clip(normal(40, 12), 18, 80))

def determiner_categorie_age(age):
    """Détermine la catégorie d'âge en fonction de l'âge."""
    if age < 30:
        return "Jeune"
    elif age < 50:
        return "Adulte"
    elif age < 65:
        return "Sénior"
    else:
        return "Âgé"

def generer_sexe():
    """Génère le sexe avec 52% de femmes et 48% d'hommes."""
    return choice(["Homme", "Femme"], p=[0.48, 0.52])

def generer_niveau_education():
    """Génère le niveau d'éducation avec une distribution réaliste."""
    niveaux = ["Sans niveau", "Fondamental", "Secondaire", "Supérieur"]
    # Ajuster ces probabilités selon les statistiques du Maroc
    probabilites = [0.25, 0.35, 0.25, 0.15]
    return choice(niveaux, p=probabilites)

def generer_annees_experience(age, niveau_education):
    """
    Génère les années d'expérience en fonction de l'âge et du niveau d'éducation.
    """
    annees_etudes = {
        "Sans niveau": 0,
        "Fondamental": 9,
        "Secondaire": 12,
        "Supérieur": uniform(15, 23)  # Entre bac+3 et doctorat
    }
    
    # Âge de début de travail = âge - années d'études
    age_debut_travail = max(16, age - annees_etudes[niveau_education])
    
    # Années d'expérience maximales = âge actuel - âge début travail
    annees_exp_max = age - age_debut_travail
    
    # Ajouter une variabilité pour tenir compte des périodes sans emploi
    return max(0, int(uniform(0, annees_exp_max)))

def generer_etat_matrimonial(age):
    """Génère l'état matrimonial en fonction de l'âge."""
    if age < 25:
        probs = [0.85, 0.15, 0.0, 0.0, 0.0]  # Majoritairement célibataire
    elif age < 35:
        probs = [0.45, 0.50, 0.05, 0.0, 0.0]  # Plus souvent marié
    elif age < 50:
        probs = [0.20, 0.65, 0.08, 0.05, 0.02]  # Majoritairement marié, quelques divorcés
    elif age < 65:
        probs = [0.10, 0.70, 0.10, 0.08, 0.02]  # Plus de veufs avec l'âge
    else:
        probs = [0.05, 0.60, 0.05, 0.28, 0.02]  # Encore plus de veufs
    
    return choice(["Célibataire", "Marié(e)", "Divorcé(e)", "Veuf(ve)", "Séparé(e)"], p=probs)

def generer_categorie_socioprofessionnelle(niveau_education, age, milieu):
    """
    Génère la catégorie socioprofessionnelle en fonction du niveau d'éducation,
    de l'âge et du milieu (urbain/rural).
    """
    categories = [
        "Groupe 1: Direction et cadres supérieurs", 
        "Groupe 2: Cadres moyens et employés",
        "Groupe 3: Inactifs (retraités, rentiers)",
        "Groupe 4: Agriculteurs et pêcheurs",
        "Groupe 5: Artisans et ouvriers qualifiés",
        "Groupe 6: Manœuvres et petits métiers"
    ]
    
    # Retraité si âge >= 60
    if age >= 60:
        return "Groupe 3: Inactifs (retraités, rentiers)"
    
    # Probabilités selon le niveau d'éducation et le milieu
    if niveau_education == "Supérieur":
        if milieu == "Urbain":
            probs = [0.40, 0.45, 0.02, 0.0, 0.10, 0.03]
        else:  # Rural
            probs = [0.15, 0.30, 0.02, 0.25, 0.18, 0.10]
    elif niveau_education == "Secondaire":
        if milieu == "Urbain":
            probs = [0.10, 0.40, 0.05, 0.01, 0.30, 0.14]
        else:  # Rural
            probs = [0.05, 0.20, 0.05, 0.30, 0.25, 0.15]
    elif niveau_education == "Fondamental":
        if milieu == "Urbain":
            probs = [0.01, 0.15, 0.08, 0.02, 0.45, 0.29]
        else:  # Rural
            probs = [0.0, 0.05, 0.08, 0.40, 0.30, 0.17]
    else:  # Sans niveau
        if milieu == "Urbain":
            probs = [0.0, 0.02, 0.15, 0.03, 0.30, 0.50]
        else:  # Rural
            probs = [0.0, 0.0, 0.15, 0.45, 0.15, 0.25]
    
    return choice(categories, p=probs)

def generer_possessions():
    """Génère les possessions avec une distribution réaliste."""
    voiture = choice([0, 1], p=[0.65, 0.35])
    logement = choice([0, 1], p=[0.40, 0.60])
    terrain = choice([0, 1], p=[0.85, 0.15])
    
    return voiture, logement, terrain

def generer_attributs_supplementaires():
    """
    Génère trois attributs supplémentaires pertinents:
    1. Nombre de personnes à charge
    2. Secteur d'activité
    3. Accès à Internet
    """
    # Nombre de personnes à charge (0 à 10)
    nb_personnes_charge = choice(range(11), p=[0.20, 0.15, 0.20, 0.15, 0.10, 0.08, 0.05, 0.03, 0.02, 0.01, 0.01])
    
    # Secteur d'activité
    secteurs = ["Public", "Privé", "Informel", "Sans emploi"]
    secteur = choice(secteurs, p=[0.15, 0.40, 0.30, 0.15])
    
    # Accès à Internet
    acces_internet = choice([0, 1], p=[0.40, 0.60])
    
    return nb_personnes_charge, secteur, acces_internet

def calculer_revenu(age, sexe, milieu, niveau_education, annees_experience, 
                    etat_matrimonial, categorie_socio, voiture, logement, terrain,
                    nb_personnes_charge, secteur, acces_internet):
    # Base selon milieu (urbain/rural)
    revenu = 26988 if milieu == "urbain" else 12862

    # Pondération par sexe (homme gagne +8% en moyenne)
    if sexe == "homme":
        revenu *= 1.08
    else:
        revenu *= 0.92

    # Âge (revenu croît avec l’âge jusqu’à ~55 ans, puis baisse légèrement)
    if age < 25:
        revenu *= 0.8
    elif age < 35:
        revenu *= 1.0
    elif age < 55:
        revenu *= 1.15
    elif age < 65:
        revenu *= 1.05
    else:
        revenu *= 0.85

    # Niveau d’éducation
    edu_coef = {
        "sans": 0.7,
        "fondamental": 0.9,
        "secondaire": 1.1,
        "superieur": 1.4
    }
    revenu *= edu_coef.get(niveau_education, 1)

    # Expérience
    revenu += min(annees_experience, 40) * 250  # Capé à 40 ans d’expérience

    # État matrimonial (petite influence)
    if etat_matrimonial == "marié":
        revenu *= 1.05
    elif etat_matrimonial in ["divorcé", "veuf"]:
        revenu *= 0.95

    # Catégorie socio-pro (groupe 1 = +70%, groupe 6 = -40%)
    coef_socio = {
        1: 1.7,
        2: 1.4,
        3: 1.1,
        4: 0.9,
        5: 0.75,
        6: 0.6
    }
    revenu *= coef_socio.get(categorie_socio, 1)

    # Possessions (voiture, logement, terrain, internet) → améliore statut
    bonus = 0
    bonus += 1200 if voiture else 0
    bonus += 1800 if logement else 0
    bonus += 1000 if terrain else 0
    bonus += 800 if acces_internet else 0
    revenu += bonus

    # Charges familiales
    revenu -= nb_personnes_charge * 500
    # Secteur d'activité (public = +10%, privé = +5%, informel = -5%)
    if secteur == "public":
        revenu *= 1.1
    elif secteur == "prive":
        revenu *= 1.05
    elif secteur == "informel":
        revenu *= 0.95
    # Ajustement final pour le revenu
    if milieu == "urbain":
        revenu = normal(revenu, ECART_TYPE_URBAIN)
    else:
        revenu = normal(revenu, ECART_TYPE_RURAL)
    # Assurer que le revenu est positif
    revenu = max(revenu, 0)
    # Ajustement pour le pourcentage de personnes gagnant moins que la moyenne
    if milieu == "urbain":
        revenu *= uniform(0.5, 1.5) if random.random() < POURCENTAGE_MOINS_MOYENNE_URBAIN else 1
    else:
        revenu *= uniform(0.5, 1.5) if random.random() < POURCENTAGE_MOINS_MOYENNE_RURAL else 1
    # Arrondir le revenu à deux décimales
    
    return round(revenu, 2)

# Génération du dataset principal

In [11]:
def generer_dataset():
    """Génère le dataset complet."""
    data = []
    
    for _ in range(NB_ENREGISTREMENTS):
        # Générer milieu de vie (urbain/rural)
        milieu = choice(["Urbain", "Rural"], p=[POURCENTAGE_URBAIN, 1-POURCENTAGE_URBAIN])
        
        # Générer caractéristiques démographiques
        age = generer_age()
        categorie_age = determiner_categorie_age(age)
        sexe = generer_sexe()
        niveau_education = generer_niveau_education()
        annees_experience = generer_annees_experience(age, niveau_education)
        etat_matrimonial = generer_etat_matrimonial(age)
        
        # Générer catégorie socioprofessionnelle
        categorie_socio = generer_categorie_socioprofessionnelle(niveau_education, age, milieu)
        
        # Générer possessions
        voiture, logement, terrain = generer_possessions()
        
        # Générer attributs supplémentaires
        nb_personnes_charge, secteur, acces_internet = generer_attributs_supplementaires()
        
        # Calculer le revenu annuel
        revenu = calculer_revenu(
            age, sexe, milieu, niveau_education, annees_experience, 
            etat_matrimonial, categorie_socio, voiture, logement, terrain,
            nb_personnes_charge, secteur, acces_internet
        )
        
        # Ajouter l'enregistrement au dataset
        data.append({
            'ID': f'MAR{_+1:06d}',
            'Age': age,
            'Categorie_Age': categorie_age,
            'Sexe': sexe,
            'Milieu': milieu,
            'Niveau_Education': niveau_education,
            'Annees_Experience': annees_experience,
            'Etat_Matrimonial': etat_matrimonial,
            'Categorie_Socioprofessionnelle': categorie_socio,
            'Possession_Voiture': voiture,
            'Possession_Logement': logement,
            'Possession_Terrain': terrain,
            'Nb_Personnes_Charge': nb_personnes_charge,
            'Secteur_Activite': secteur,
            'Acces_Internet': acces_internet,
            'Revenu_Annuel': revenu
        })
    
    return pd.DataFrame(data)

#  Injection de valeurs manquantes et aberrantes

In [12]:
def injecter_valeurs_manquantes(df, pourcentage=0.05):
    """Injecte des valeurs manquantes dans le dataset."""
    # Colonnes où nous pouvons injecter des valeurs manquantes
    colonnes_cibles = [
        'Niveau_Education', 'Annees_Experience', 'Etat_Matrimonial',
        'Possession_Voiture', 'Possession_Logement', 'Possession_Terrain',
        'Nb_Personnes_Charge', 'Secteur_Activite', 'Acces_Internet'
    ]
    
    for colonne in colonnes_cibles:
        # Sélectionner aléatoirement des indices pour mettre à NaN
        masque = np.random.choice([True, False], size=len(df), p=[pourcentage, 1-pourcentage])
        df.loc[masque, colonne] = np.nan
    
    return df

def injecter_valeurs_aberrantes(df, pourcentage=0.02):
    """Injecte des valeurs aberrantes dans le dataset."""
    # Valeurs aberrantes pour l'âge
    masque_age = np.random.choice([True, False], size=len(df), p=[pourcentage, 1-pourcentage])
    df.loc[masque_age, 'Age'] = np.random.choice([5, 10, 15, 100, 120], size=sum(masque_age))
    
    # Valeurs aberrantes pour les années d'expérience (> âge - 18)
    masque_exp = np.random.choice([True, False], size=len(df), p=[pourcentage, 1-pourcentage])
    df.loc[masque_exp, 'Annees_Experience'] = df.loc[masque_exp, 'Age'] + np.random.randint(1, 20, size=sum(masque_exp))
    
    # Valeurs aberrantes pour le revenu
    masque_revenu = np.random.choice([True, False], size=len(df), p=[pourcentage, 1-pourcentage])
    df.loc[masque_revenu, 'Revenu_Annuel'] = np.random.choice(
        [-10000, -5000, 0, 1000000, 2000000, 5000000], 
        size=sum(masque_revenu)
    )
    
    return df

def ajouter_colonnes_redondantes(df):
    """Ajoute des colonnes redondantes au dataset."""
    # Age en mois (redondant avec Age)
    df['Age_Mois'] = df['Age'] * 12 + np.random.randint(-2, 3, size=len(df))
    
    # Indicateur binaire pour éducation supérieure (redondant avec Niveau_Education)
    df['Education_Superieure'] = (df['Niveau_Education'] == 'Supérieur').astype(int)
    
    # Indicateur pour niveau de richesse (redondant avec les possessions)
    df['Indice_Richesse'] = df['Possession_Voiture'] + df['Possession_Logement'] + df['Possession_Terrain']
    
    return df

def ajouter_colonnes_non_pertinentes(df):
    """Ajoute des colonnes non pertinentes au dataset."""
    # Couleur préférée (non corrélée au revenu)
    couleurs = ['Rouge', 'Bleu', 'Vert', 'Jaune', 'Noir', 'Blanc', 'Orange']
    df['Couleur_Preferee'] = np.random.choice(couleurs, size=len(df))
    
    # Code postal aléatoire
    df['Code_Postal'] = np.random.randint(10000, 99999, size=len(df))
    
    # Identifiant alpha-numérique aléatoire
    lettres = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    df['ID_Aleatoire'] = [''.join(np.random.choice(list(lettres), 3)) + str(np.random.randint(100, 999)) 
                          for _ in range(len(df))]
    
    return df

# Vérification et finalisation du dataset

In [13]:
def verifier_statistiques(df):
    """Vérifie que le dataset respecte les statistiques requises."""
    print("Analyse du dataset généré:")
    print(f"Nombre total d'enregistrements: {len(df)}")
    
    # Répartition urbain/rural
    urbain_pct = (df['Milieu'] == 'Urbain').mean() * 100
    print(f"Pourcentage urbain: {urbain_pct:.1f}%")
    
    # Revenu moyen
    revenu_moyen_total = df['Revenu_Annuel'].mean()
    revenu_moyen_urbain = df[df['Milieu'] == 'Urbain']['Revenu_Annuel'].mean()
    revenu_moyen_rural = df[df['Milieu'] == 'Rural']['Revenu_Annuel'].mean()
    
    print(f"Revenu moyen total: {revenu_moyen_total:.2f} DH")
    print(f"Revenu moyen urbain: {revenu_moyen_urbain:.2f} DH")
    print(f"Revenu moyen rural: {revenu_moyen_rural:.2f} DH")
    
    # Pourcentage sous la moyenne
    pct_sous_moyenne = (df['Revenu_Annuel'] < REVENU_MOYEN).mean() * 100
    pct_sous_moyenne_urbain = (df[(df['Milieu'] == 'Urbain')]['Revenu_Annuel'] < REVENU_MOYEN_URBAIN).mean() * 100
    pct_sous_moyenne_rural = (df[(df['Milieu'] == 'Rural')]['Revenu_Annuel'] < REVENU_MOYEN_RURAL).mean() * 100
    
    print(f"Pourcentage sous la moyenne (total): {pct_sous_moyenne:.1f}%")
    print(f"Pourcentage sous la moyenne (urbain): {pct_sous_moyenne_urbain:.1f}%")
    print(f"Pourcentage sous la moyenne (rural): {pct_sous_moyenne_rural:.1f}%")
    
    # Valeurs manquantes
    print(f"\nNombre de valeurs manquantes par colonne:")
    print(df.isna().sum())
    
    return {
        'revenu_moyen_total': revenu_moyen_total,
        'revenu_moyen_urbain': revenu_moyen_urbain,
        'revenu_moyen_rural': revenu_moyen_rural,
        'pct_sous_moyenne': pct_sous_moyenne / 100,
        'pct_sous_moyenne_urbain': pct_sous_moyenne_urbain / 100,
        'pct_sous_moyenne_rural': pct_sous_moyenne_rural / 100
    }

def ajuster_revenus(df, stats):
    """
    Ajuste les revenus pour correspondre aux statistiques requises.
    Cette fonction est utilisée si les statistiques sont trop éloignées des cibles.
    """
    # Facteurs de correction
    facteur_total = REVENU_MOYEN / stats['revenu_moyen_total']
    facteur_urbain = REVENU_MOYEN_URBAIN / stats['revenu_moyen_urbain']
    facteur_rural = REVENU_MOYEN_RURAL / stats['revenu_moyen_rural']
    
    # Appliquer les facteurs
    df.loc[df['Milieu'] == 'Urbain', 'Revenu_Annuel'] *= facteur_urbain
    df.loc[df['Milieu'] == 'Rural', 'Revenu_Annuel'] *= facteur_rural
    
    # Redistribuer pour atteindre les pourcentages sous la moyenne
    df_urbain = df[df['Milieu'] == 'Urbain'].copy()
    df_rural = df[df['Milieu'] == 'Rural'].copy()
    
    # Fonction pour redistribuer
    def redistribuer(df_subset, cible_moyenne, cible_pct_sous):
        tri = df_subset.sort_values('Revenu_Annuel')
        index_seuil = int(len(tri) * cible_pct_sous)
        valeur_seuil = tri.iloc[index_seuil]['Revenu_Annuel']
        
        # Ajustement des revenus sous le seuil
        df_subset.loc[df_subset['Revenu_Annuel'] < valeur_seuil, 'Revenu_Annuel'] *= 0.95
        # Ajustement des revenus au-dessus du seuil
        df_subset.loc[df_subset['Revenu_Annuel'] >= valeur_seuil, 'Revenu_Annuel'] *= 1.05
        
        return df_subset
    
    df_urbain = redistribuer(df_urbain, REVENU_MOYEN_URBAIN, POURCENTAGE_MOINS_MOYENNE_URBAIN)
    df_rural = redistribuer(df_rural, REVENU_MOYEN_RURAL, POURCENTAGE_MOINS_MOYENNE_RURAL)
    
    # Reconstituer le DataFrame
    df_ajuste = pd.concat([df_urbain, df_rural])
    
    return df_ajuste

def plot_distribution_revenu(df):
    """Crée un histogramme de la distribution des revenus."""
    plt.figure(figsize=(12, 6))
    
    # Distribution globale
    plt.subplot(1, 3, 1)
    sns.histplot(df['Revenu_Annuel'], kde=True)
    plt.axvline(x=REVENU_MOYEN, color='red', linestyle='--', label=f'Moyenne: {REVENU_MOYEN}')
    plt.title('Distribution des revenus')
    plt.xlabel('Revenu annuel (DH)')
    plt.ylabel('Fréquence')
    plt.legend()
    
    # Distribution urbaine
    plt.subplot(1, 3, 2)
    sns.histplot(df[df['Milieu'] == 'Urbain']['Revenu_Annuel'], kde=True, color='blue')
    plt.axvline(x=REVENU_MOYEN_URBAIN, color='red', linestyle='--', label=f'Moyenne: {REVENU_MOYEN_URBAIN}')
    plt.title('Distribution urbaine')
    plt.xlabel('Revenu annuel (DH)')
    plt.legend()
    
    # Distribution rurale
    plt.subplot(1, 3, 3)
    sns.histplot(df[df['Milieu'] == 'Rural']['Revenu_Annuel'], kde=True, color='green')
    plt.axvline(x=REVENU_MOYEN_RURAL, color='red', linestyle='--', label=f'Moyenne: {REVENU_MOYEN_RURAL}')
    plt.title('Distribution rurale')
    plt.xlabel('Revenu annuel (DH)')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig('distribution_revenus.png')
    plt.close()

# Fonction principale et exécution

In [14]:
def main():
    """Fonction principale pour générer le dataset."""
    print("Génération du dataset de revenus des Marocains...")
    
    # Générer le dataset de base
    df = generer_dataset()
    
    # Vérifier les statistiques initiales
    stats_initiales = verifier_statistiques(df)
    
    # Ajuster les revenus si nécessaire
    if (abs(stats_initiales['revenu_moyen_total'] - REVENU_MOYEN) / REVENU_MOYEN > 0.05 or
        abs(stats_initiales['pct_sous_moyenne'] - POURCENTAGE_MOINS_MOYENNE) / POURCENTAGE_MOINS_MOYENNE > 0.05):
        print("\nAjustement des revenus pour correspondre aux statistiques cibles...")
        df = ajuster_revenus(df, stats_initiales)
        stats_finales = verifier_statistiques(df)
    else:
        stats_finales = stats_initiales
    
    # Injecter des valeurs manquantes, aberrantes et des colonnes redondantes/non pertinentes
    print("\nInjection des valeurs manquantes...")
    df = injecter_valeurs_manquantes(df)
    
    print("Injection des valeurs aberrantes...")
    df = injecter_valeurs_aberrantes(df)
    
    print("Ajout de colonnes redondantes...")
    df = ajouter_colonnes_redondantes(df)
    
    print("Ajout de colonnes non pertinentes...")
    df = ajouter_colonnes_non_pertinentes(df)
    
    # Créer un graphique de la distribution des revenus
    plot_distribution_revenu(df)
    
    # Enregistrer le dataset
    df.to_csv(r"C:\Users\salah\OneDrive\Documents\Prédiction du revenu annuel d’un marocain\data\raw\dataset_revenu_marocains.csv", index=False)
    print("\nDataset enregistré avec succès dans 'dataset_revenu_marocains.csv'")
    print(f"Nombre total d'enregistrements: {len(df)}")

if __name__ == "__main__":
    main()

Génération du dataset de revenus des Marocains...
Analyse du dataset généré:
Nombre total d'enregistrements: 40000
Pourcentage urbain: 65.0%
Revenu moyen total: 14362.35 DH
Revenu moyen urbain: 14381.26 DH
Revenu moyen rural: 14327.23 DH
Pourcentage sous la moyenne (total): 82.1%
Pourcentage sous la moyenne (urbain): 92.0%
Pourcentage sous la moyenne (rural): 47.9%

Nombre de valeurs manquantes par colonne:
ID                                0
Age                               0
Categorie_Age                     0
Sexe                              0
Milieu                            0
Niveau_Education                  0
Annees_Experience                 0
Etat_Matrimonial                  0
Categorie_Socioprofessionnelle    0
Possession_Voiture                0
Possession_Logement               0
Possession_Terrain                0
Nb_Personnes_Charge               0
Secteur_Activite                  0
Acces_Internet                    0
Revenu_Annuel                     0
dtype: int64
